In [ ]:
import pandas as pd
import numpy as np
file_path = "raw_data/merged_data.csv"
chunk_size = 1000000
for chunk in pd.read_csv(file_path, chunksize=chunk_size,sep='|'):
    print(f"Processing chunk with {len(chunk)} rows")
    print(chunk.info())
df=pd.read_csv(file_path, nrows=1000000,sep='|')
df.head(20)
chunks = pd.read_csv(file_path, chunksize=chunk_size,sep='|')

total_rows = 0
nan_count = None

In [13]:
df = pd.read_csv(file_path, sep='|', low_memory=False)

In [14]:
df_cleaned = df.drop_duplicates()
df_cleaned.reset_index(drop=True, inplace=True)

In [15]:
drop_cols = ['uid', 'history', 'tunnel_parents', 'detailed-label', 'local_resp', 'local_orig', 'missed_bytes']
df_cleaned.drop(columns=drop_cols, inplace=True, errors='ignore')


In [16]:
df_cleaned.dropna(subset=['label', 'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes'], inplace=True)

In [17]:
for col in ['duration', 'orig_bytes', 'resp_bytes']:
    df_cleaned[col] = df_cleaned[col].replace('-', np.nan)  # Replace '-' with NaN
    df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')  # Convert to numeric
    df_cleaned[col].fillna(0, inplace=True)

/var/folders/wx/l8wg57xn4zg_6jvfv7d2rly40000gn/T/ipykernel_98591/970632311.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned[col].fillna(0, inplace=True)


In [18]:
df_cleaned['proto'] = df_cleaned['proto'].astype('category').cat.codes
df_cleaned['service'] = df_cleaned['service'].astype('category').cat.codes


In [19]:
df_cleaned['label'] = df_cleaned['label'].apply(lambda x: 0 if x == 'Benign' else 1)

In [20]:
features = ['proto', 'service', 'duration', 'orig_bytes', 'resp_bytes']
X = df_cleaned[features]
y = df_cleaned['label']

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    stratify=y,
                                                    random_state=42)

In [23]:
clf = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1)

clf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [24]:
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7616057252667859
Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.52      0.60   2634047
           1       0.77      0.89      0.83   4869254

    accuracy                           0.76   7503301
   macro avg       0.75      0.71      0.72   7503301
weighted avg       0.76      0.76      0.75   7503301

